In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,classification_report,roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [2]:
df=pd.read_csv('processed_outliers.csv')
print('Data shape:',df.shape)

Data shape: (1159, 22)


In [3]:
list(enumerate(df.columns))

[(0, 'w/b'),
 (1, 'Water'),
 (2, 'Cement type'),
 (3, 'Cement'),
 (4, 'Slag'),
 (5, 'Fly ash'),
 (6, 'Silica fume'),
 (7, 'Lime filler'),
 (8, 'FA'),
 (9, 'CA'),
 (10, 'Plasticizer'),
 (11, 'Superplasticizer'),
 (12, 'Air entraining'),
 (13, 'Comp. str. test age'),
 (14, 'Compressive strength'),
 (15, 'Air content'),
 (16, 'Spreed'),
 (17, 'Slump'),
 (18, 'Fresh density'),
 (19, 'Dry  density'),
 (20, 'Migration test age'),
 (21, 'Migration resistance')]

1-hot encoding

In [4]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2])], remainder='passthrough')
X=ct.fit_transform(df)
# remove dummy variable and output
y=X[:,-1]
X=X[:,1:-1]
print(X.shape)
print(y.shape)

(1159, 30)
(1159,)


SVM Classifier

In [5]:
param_grid={
    'kernel': ['rbf','sigmoid','linear','poly'],
    'C': list(np.logspace(-3,3,7)) + [200],
    'gamma': list(np.logspace(-3,3,7)) + ['scale','auto'],
    'degree': [3,4,5]
}

splits=list(StratifiedKFold(shuffle=True,random_state=0).split(X,y))
train_index, test_index = splits[0]
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

def train_svc(X_train,y_train):
    clf = SVC(random_state=0,max_iter=int(1e9))
    grid=GridSearchCV(clf,param_grid,cv=5,n_jobs=3,scoring='accuracy')
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    print(grid.best_score_)
    return grid

def cv_scores(X,y,model):
    acc_test=[]
    acc_train=[]
    for train_index, test_index in StratifiedKFold(shuffle=True,random_state=1).split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        model.fit(X_train,y_train)
        acc_test.append(accuracy_score(y_test,model.predict(X_test)))
        acc_train.append(accuracy_score(y_train,model.predict(X_train)))
    print('Train acc:',np.mean(acc_train))
    print('Test acc:',np.mean(acc_test))

In [18]:
grid=train_svc(X_train,y_train)

{'C': 100.0, 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf'}
0.7876663760534728


In [6]:
best_params={'C': 100.0, 'gamma': 0.1, 'kernel': 'rbf'}
best_model=SVC(random_state=0,max_iter=int(1e9),**best_params,probability=True)
best_model.fit(X_train,y_train)
print(classification_report(y_test,best_model.predict(X_test)))
roc_auc_score(y_test,best_model.predict_proba(X_test),multi_class='ovr')

              precision    recall  f1-score   support

         0.0       0.91      0.85      0.88        46
         1.0       0.91      0.87      0.89        47
         2.0       0.79      0.83      0.81        46
         3.0       0.88      0.89      0.88        47
         4.0       0.96      1.00      0.98        46

    accuracy                           0.89       232
   macro avg       0.89      0.89      0.89       232
weighted avg       0.89      0.89      0.89       232



0.9732256532143136

Removing cement type

In [6]:
X=df.drop('Cement type',axis=1).iloc[:,:-1].values
y=df.iloc[:,-1].values
print(X.shape)
print(y.shape)

(1159, 20)
(1159,)


In [7]:
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
grid=train_svc(X_train,y_train)

{'C': 200, 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf'}
0.8275210694565533


In [8]:
best_params={'C': 200, 'gamma': 0.1, 'kernel': 'rbf'}
best_model=SVC(random_state=0,max_iter=int(1e9),**best_params,probability=True)
best_model.fit(X_train,y_train)
print(classification_report(y_test,best_model.predict(X_test)))
roc_auc_score(y_test,best_model.predict_proba(X_test),multi_class='ovr')

              precision    recall  f1-score   support

           0       0.93      0.87      0.90        46
           1       0.87      0.85      0.86        47
           2       0.85      0.85      0.85        46
           3       0.90      0.94      0.92        47
           4       0.96      1.00      0.98        46

    accuracy                           0.90       232
   macro avg       0.90      0.90      0.90       232
weighted avg       0.90      0.90      0.90       232



0.9745144568638346

In [9]:
accuracy_score(y_test,best_model.predict(X_test))

0.9008620689655172